In [1]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
import utils
import os
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
mixed_precision.set_global_policy('mixed_float16')

2024-03-16 14:50:37.754083: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-16 14:50:37.776556: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 14:50:37.776583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 14:50:37.776606: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 14:50:37.781656: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-16 14:50:40.977833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-16 14:50:41.010211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-16 14:50:41.010464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 299, 299
BATCH_SIZE = 32
labels = sorted(os.listdir(train_data_path))
labels

['Fulvous whistling duck',
 'acorn_woodpecker',
 'american_flamingo',
 'american_three_toed_woodpecker',
 'american_wigeon',
 'amethyst_throated_mountain_gem',
 "anna's_hummingbird",
 'baikal_teal',
 'barrows_goldeneye',
 'black_bellied_whistling_duck',
 'black_chinned_hummingbird',
 'black_vulture',
 'boreal_owl',
 'brant_goose',
 'broad_billed_hummingbird',
 'broad_tailed_hummingbird',
 'buffle_head',
 'california_condor',
 'california_quail',
 'calliope_hummingbird',
 'canvas_back',
 'chukar_partridge',
 'cinnamon_teal',
 'common_goldeneye',
 'costas_hummingbird',
 'downy_woodpecker',
 'emperor_goose',
 'gray_partridge',
 'greater_prairie_chicken',
 'greater_sage_grouse',
 'greater_white_fronted_goose',
 'greylag_goose',
 'harlequin_duck',
 'hooded_merganser',
 'king_eider',
 "lewi's_woodpecker",
 'limpkin',
 'mallard_duck',
 'mexican_violetear',
 'northern_flicker',
 'northern_shoveler',
 'pileated_woodpecker',
 'pink_footed_goose',
 'red_bellied_woodpecker',
 'red_breasted_mergans

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation'
)


Found 9404 files belonging to 68 classes.
Using 7524 files for training.


2024-02-08 04:05:35.818867: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 04:05:35.819084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 04:05:35.819225: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 4069 files belonging to 68 classes.
Using 813 files for validation.


In [4]:
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))

In [18]:
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False
augmentation_layer = utils.return_data_aug_layer_for_eff_net()
inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
#x = augmentation_layer(inputs)
x = base_model(inputs, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(len(labels), activation='softmax', dtype=tf.float32)(x)
model_inception_1 = tf.keras.Model(inputs, outputs)

In [15]:
model_inception_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_8 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0   

In [21]:


model_inception_1.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.75),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

# Train the model
history = model_inception_1.fit(train_dataset,
                              steps_per_epoch=len(train_dataset),
                              validation_data=validation_dataset,
                              validation_steps=len(validation_dataset),
                              epochs=30,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer])

# Save the trained model
model_inception_1.save('model_trained_3class.hdf5')

Epoch 1/30


/tmp/ipykernel_5831/2818544448.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model_inception_1.fit_generator(train_dataset,


235/236 [============================>.] - ETA: 0s - loss: 2.2300 - accuracy: 0.3294
Epoch 1: val_loss improved from inf to 1.52937, saving model to best_model_3class.hdf5
236/236 [==============================] - 14s 48ms/step - loss: 2.2302 - accuracy: 0.3292 - val_loss: 1.5294 - val_accuracy: 0.5769
Epoch 2/30
235/236 [============================>.] - ETA: 0s - loss: 1.8813 - accuracy: 0.4242
Epoch 2: val_loss improved from 1.52937 to 1.31440, saving model to best_model_3class.hdf5
236/236 [==============================] - 11s 46ms/step - loss: 1.8815 - accuracy: 0.4241 - val_loss: 1.3144 - val_accuracy: 0.6199
Epoch 3/30
234/236 [============================>.] - ETA: 0s - loss: 1.6992 - accuracy: 0.4708
Epoch 3: val_loss improved from 1.31440 to 1.11862, saving model to best_model_3class.hdf5
236/236 [==============================] - 11s 46ms/step - loss: 1.6985 - accuracy: 0.4710 - val_loss: 1.1186 - val_accuracy: 0.6654
Epoch 4/30
234/236 [============================>.] - E

In [22]:
history2 = model_inception_1.fit(train_dataset,
                              steps_per_epoch=len(train_dataset),
                              validation_data=validation_dataset,
                              validation_steps=len(validation_dataset),
                              epochs=100,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer],
                                 initial_epoch=history.epoch[-1])

Epoch 30/100
235/236 [============================>.] - ETA: 0s - loss: 0.5533 - accuracy: 0.8090
Epoch 30: val_loss did not improve from 0.41393
236/236 [==============================] - 11s 44ms/step - loss: 0.5531 - accuracy: 0.8091 - val_loss: 0.4290 - val_accuracy: 0.8536
Epoch 31/100
235/236 [============================>.] - ETA: 0s - loss: 0.5389 - accuracy: 0.8199
Epoch 31: val_loss did not improve from 0.41393
236/236 [==============================] - 11s 44ms/step - loss: 0.5387 - accuracy: 0.8200 - val_loss: 0.4242 - val_accuracy: 0.8635
Epoch 32/100
234/236 [============================>.] - ETA: 0s - loss: 0.5428 - accuracy: 0.8146
Epoch 32: val_loss improved from 0.41393 to 0.41088, saving model to best_model_3class.hdf5
236/236 [==============================] - 11s 45ms/step - loss: 0.5425 - accuracy: 0.8149 - val_loss: 0.4109 - val_accuracy: 0.8610
Epoch 33/100
235/236 [============================>.] - ETA: 0s - loss: 0.4952 - accuracy: 0.8347
Epoch 33: val_loss im

In [5]:

base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False
augmentation_layer = utils.return_data_aug_layer_for_eff_net()
inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
#x = augmentation_layer(inputs)
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(len(labels), activation='softmax', dtype=tf.float32)(x)
model_inception_1 = tf.keras.Model(inputs, outputs)
for layer in model_inception_1.layers[:-50]:
    layer.trainable = False
for layer in model_inception_1.layers[-50:]:
    layer.trainable = True


In [6]:
model_inception_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                             

In [9]:
model_inception_1.compile(optimizer=SGD(learning_rate=0.001, momentum=0.5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

# Train the model
history = model_inception_1.fit(train_dataset,
                              steps_per_epoch=len(train_dataset),
                              validation_data=validation_dataset,
                              validation_steps=len(validation_dataset),
                              epochs=30,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer])

Epoch 1/30
235/236 [============================>.] - ETA: 0s - loss: 4.1290 - accuracy: 0.0338
Epoch 1: val_loss improved from inf to 3.95660, saving model to best_model_3class.hdf5
236/236 [==============================] - 33s 113ms/step - loss: 4.1289 - accuracy: 0.0338 - val_loss: 3.9566 - val_accuracy: 0.1501
Epoch 2/30
235/236 [============================>.] - ETA: 0s - loss: 3.8985 - accuracy: 0.0767
Epoch 2: val_loss improved from 3.95660 to 3.48004, saving model to best_model_3class.hdf5
236/236 [==============================] - 26s 111ms/step - loss: 3.8982 - accuracy: 0.0768 - val_loss: 3.4800 - val_accuracy: 0.2448
Epoch 3/30
236/236 [==============================] - ETA: 0s - loss: 3.5078 - accuracy: 0.1381

2024-02-08 04:10:35.719667: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-02-08 04:10:35.720100: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-02-08 04:10:35.720114: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-02-08 04:10:35.720236: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-02-08 04:10:35.720252: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-02-08


Epoch 3: val_loss improved from 3.48004 to 2.74693, saving model to best_model_3class.hdf5
236/236 [==============================] - 26s 111ms/step - loss: 3.5078 - accuracy: 0.1381 - val_loss: 2.7469 - val_accuracy: 0.3530
Epoch 4/30
235/236 [============================>.] - ETA: 0s - loss: 2.9811 - accuracy: 0.2355
Epoch 4: val_loss improved from 2.74693 to 2.20364, saving model to best_model_3class.hdf5
236/236 [==============================] - 26s 110ms/step - loss: 2.9811 - accuracy: 0.2355 - val_loss: 2.2036 - val_accuracy: 0.4982
Epoch 5/30
235/236 [============================>.] - ETA: 0s - loss: 2.4912 - accuracy: 0.3499
Epoch 5: val_loss improved from 2.20364 to 1.44752, saving model to best_model_3class.hdf5
236/236 [==============================] - 26s 111ms/step - loss: 2.4910 - accuracy: 0.3501 - val_loss: 1.4475 - val_accuracy: 0.6913
Epoch 6/30
235/236 [============================>.] - ETA: 0s - loss: 2.0210 - accuracy: 0.4570
Epoch 6: val_loss improved from 1.44

In [13]:
model_inception_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                             

In [14]:

base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False



In [15]:
model_inception_1.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.75),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

# Train the model
history = model_inception_1.fit(train_dataset,
                              steps_per_epoch=len(train_dataset),
                              validation_data=validation_dataset,
                              validation_steps=len(validation_dataset),
                              epochs=30,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer])

Epoch 1/30
235/236 [============================>.] - ETA: 0s - loss: 0.1354 - accuracy: 0.9588
Epoch 1: val_loss improved from inf to 0.31408, saving model to best_model_3class.hdf5
236/236 [==============================] - 15s 52ms/step - loss: 0.1353 - accuracy: 0.9588 - val_loss: 0.3141 - val_accuracy: 0.9446
Epoch 2/30
235/236 [============================>.] - ETA: 0s - loss: 0.1362 - accuracy: 0.9533
Epoch 2: val_loss improved from 0.31408 to 0.30436, saving model to best_model_3class.hdf5
236/236 [==============================] - 12s 50ms/step - loss: 0.1362 - accuracy: 0.9533 - val_loss: 0.3044 - val_accuracy: 0.9471
Epoch 3/30
235/236 [============================>.] - ETA: 0s - loss: 0.1257 - accuracy: 0.9596
Epoch 3: val_loss did not improve from 0.30436
236/236 [==============================] - 12s 48ms/step - loss: 0.1257 - accuracy: 0.9595 - val_loss: 0.3052 - val_accuracy: 0.9446
Epoch 4/30
235/236 [============================>.] - ETA: 0s - loss: 0.1113 - accuracy:

KeyboardInterrupt: 